In [9]:
import json
import boto3
from typing import Dict

SECRET_NAME = "ds4a-project-dev-db-connection-secret"


def get_secret(aws: boto3.Session, secret_name: str) -> Dict[str, str]:
    sm = aws.client("secretsmanager")
    response = sm.get_secret_value(SecretId=secret_name)
    secret = response["SecretString"]
    return json.loads(secret)


aws = boto3.Session()
params = get_secret(aws, SECRET_NAME)
params.keys()


dict_keys(['password', 'dbname', 'engine', 'port', 'dbInstanceIdentifier', 'host', 'username'])

In [10]:
import psycopg2


conn = psycopg2.connect(
    host=params["host"],
    database=params["dbname"],
    user=params["username"],
    password=params["password"],
)

cur = conn.cursor()
cur.execute("SELECT version()")
cur.fetchone()


('PostgreSQL 13.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)

In [40]:
from psycopg2.extras import RealDictCursor

query = """--sql
    SELECT
        name,
        code,
        CASE WHEN latitude = 'NaN' THEN 0 ELSE latitude END AS lat,
        CASE WHEN longitude = 'NaN' THEN 0 ELSE longitude END AS lon
    FROM municipalities
    WHERE department_id = 5
    ORDER BY lat ASC
    LIMIT 5
"""

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    res = cursor.fetchall()
    conn.commit()
except:
    res = []
    conn.rollback()

res

[RealDictRow([('name', 'Carolina'),
              ('code', 5150),
              ('lat', Decimal('0')),
              ('lon', Decimal('0'))]),
 RealDictRow([('name', 'Yondó'),
              ('code', 5893),
              ('lat', Decimal('0')),
              ('lon', Decimal('0'))]),
 RealDictRow([('name', 'San Vicente'),
              ('code', 5674),
              ('lat', Decimal('0')),
              ('lon', Decimal('0'))]),
 RealDictRow([('name', 'Caramanta'),
              ('code', 5145),
              ('lat', Decimal('5.54853427166')),
              ('lon', Decimal('-75.6438712654'))]),
 RealDictRow([('name', 'Jardín'),
              ('code', 5364),
              ('lat', Decimal('5.59753984677')),
              ('lon', Decimal('-75.8189807815'))])]

In [5]:
# drop_statement = """--sql
#     DROP TABLE municipalities
#     """

# try:
#     cur.execute(drop_statement)
#     conn.commit()
# except Exception as e:
#     print(e)
#     conn.rollback()


In [6]:
from psycopg2.extras import RealDictCursor

query = """--sql
SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'
"""

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    res = cursor.fetchall()
    conn.commit()
except:
    res = None
    conn.rollback()

res


[RealDictRow([('table_name', 'departments')]),
 RealDictRow([('table_name', 'departmentspopulationhistory')])]

In [8]:
conn.close()